# Computational Finance Fundamentals
## Computing the expected returns and variances for a portfolio having N number of stocks.
reference:  https://blog.quantinsti.com/portfolio-analysis-calculating-risk-returns/

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt

Let ri be the expected return on the stock and rx be any return having a probability of px. The expected return, ri, can be computed using the below equation.

![](https://d1rwhvwstyk9gu.cloudfront.net/2018/01/expected-return-equation.jpg)

In [2]:
px = np.random.rand(10)
px = px/sum(px)
print("Probabilities summing up to: ", sum(px))
rx = np.random.rand(10)*12
ri_list = px*rx
print("Expected Return: ", round(sum(ri_list), 3), "%")
df = pd.DataFrame({'Probability (px)': px, "Probable Return (rx)": rx, "px*rx": ri_list})
df.head()

Probabilities summing up to:  1.0
Expected Return:  4.981 %


,Probability (px),Probable Return (rx),px*rx
0,0.083114,3.252859,0.270359
1,0.107589,4.991793,0.537064
2,0.067839,2.918375,0.197979
3,0.003088,11.779836,0.036380
4,0.012801,0.536138,0.006863


Risk (or variance) on a single stock

![](https://d1rwhvwstyk9gu.cloudfront.net/2018/01/Variance-of-the-return-on-stock.jpg)

In [3]:
ri = sum(ri_list)
variance = sum(px*(rx-ri)**2)
variance

10.226466195592437

In [4]:
std = np.sqrt(variance)
std

3.1978846438845223

Expected return on an N-stock portfolio

The expected return from ith stock is ri. The expected return on the portfolio will then be:

![](https://d1rwhvwstyk9gu.cloudfront.net/2018/01/n-stock-portfolio.jpg)

The weight of any stock is the ratio of the amount invested in that stock to the total amount invested.

In [5]:
invested_amount = np.random.rand(10)*132
weights = invested_amount/sum(invested_amount)
print("Weights summing up toL ", sum(px))

Weights summing up toL  1.0


## Portfolio Optimization Using Monte Carlo Simulation
Reference Guide: https://blog.quantinsti.com/portfolio-optimization-maximum-return-risk-ratio-python/

In [11]:
!pip install yahoofinancials
!pip install fix_yahoo_finance
!pip install yfinance

     |████████████████████████████████| 6.3 MB 19.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=c846a27a5f27289250bb3b18dc40a175338c2dff480580764797d0dc2080cd13
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [15]:
!pip install pandas-datareader

In [ ]:
# Fetch data from yahoo and save under DataFrame named 'data'
stock = ['BAC', 'GS', 'JPM', 'MS']
data = web.DataReader(stock, data_source="yahoo",start='12/01/2019',end='12/31/2019')['Adj Close']
# Arrange the data in ascending order
data=data.iloc[::-1]
data.round(2)

In [ ]:
# Compute stock returns and print the returns in percentage format
stock_ret = data.pct_change()
stock_ret.round(4)*100

In [ ]:
# Calculate mean returns and covariances of all four the stocks
mean_returns = stock_ret.mean()
cov_matrix = stock_ret.cov()

In [ ]:

# Set the number of iterations to 10000 and define an array to hold 
# the simulation results; initially set to all zeros
num_iterations = 10000
simulation_res = np.zeros((4+len(stock)-1,num_iterations))

for i in range(num_iterations):
    # Select random weights and normalize to set the sum to 1
    weights = np.array(np.random.random(4))
    weights /= np.sum(weights)
    # Calculate the return and standard deviation for every step
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))
    # Store all the results in a defined array
    simulation_res[0,i] = portfolio_return
    simulation_res[1,i] = portfolio_std_dev
    # Store all the results in a defined array
    simulation_res[0,i] = portfolio_return
    simulation_res[1,i] = portfolio_std_dev
    # Save the weights in the array
    for j in range(len(weights)):
        simulation_res[j+3,i] = weights[j]

In [ ]:
sim_frame = pd.DataFrame(simulation_res.T,
                         columns=['ret','stdev','sharpe',
                                  stock[0],stock[1],stock[2],stock[3]])
sim_frame.head(5)

In [ ]:
# Spot the position of the portfolio with highest Sharpe Ratio
max_sharpe = sim_frame.iloc[sim_frame['sharpe'].idxmax()]
# Spot the position of the portfolio with minimum Standard Deviation
min_std = sim_frame.iloc[sim_frame['stdev'].idxmin()]
print("The portfolio for max Sharpe Ratio:\n" + str(max_sharpe))
print("The portfolio for min risk:\n" + str(min_std))

In [ ]:
# Create a scatter plot coloured by various Sharpe Ratios with 
# standard deviation on the x-axis and returns on the y-axis
plt.scatter(sim_frame.stdev,sim_frame.ret,c=sim_frame.sharpe,cmap='RdYlBu')
# Plot a red star to highlight position of the portfolio with highest Sharpe Ratio
plt.scatter(max_sharpe[1],max_sharpe[0],marker=(5,1,0),color='r',s=600)
# Plot a blue star to highlight position of the portfolio with minimum Variance
plt.scatter(min_std[1],min_std[0],marker=(5,1,0),color='b',s=600)
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.ylim(-0.003,0.0005)
plt.xlim(0.008,0.012)
plt.show()